# Test THREDDS using Magpie Authentication/Authorization against Twitcher proxy.

## Configuration parameters

In [ ]:
from __future__ import print_function
import os
import uuid
import requests
print("Setup configuration parameters...")

VERIFY_SSL = True if "DISABLE_VERIFY_SSL" not in os.environ else False
PAVICS_HOST = os.getenv("PAVICS_HOST", "").rstrip("/")
MAGPIE_URL = PAVICS_HOST + os.getenv("MAGPIE_ROOT", "/magpie")
TWITCHER_PROXY = os.getenv("TWITCHER_PROXY", "/twitcher/ows/proxy")
TWITCHER_URL = PAVICS_HOST + TWITCHER_PROXY
THREDDS_SERVICE = os.getenv("THREDDS_SERVICE", "thredds")
THREDDS_URL = "{}/{}".format(TWITCHER_URL, THREDDS_SERVICE)

HEADERS = {"Accept": "application/json", "Content-Type": "application/json"}

MAGPIE_TEST_ADMIN_USERNAME = os.getenv("MAGPIE_TEST_ADMIN_USERNAME")
MAGPIE_TEST_ADMIN_PASSWORD = os.getenv("MAGPIE_TEST_ADMIN_PASSWORD")
MAGPIE_ANONYMOUS_GROUP = os.getenv("MAGPIE_ANONYMOUS_GROUP", "anonymous")

print("  Will use Magpie URL:       [{}]".format(MAGPIE_URL))
print("  Will use Twitcher URL:     [{}]".format(TWITCHER_URL))
print("  Will use THREDDS service:  [{}]".format(THREDDS_SERVICE))

## Validate test admin user can have needed access to adjust permissions and that anonymous group is valid

In [ ]:
print("Validate that user has needed admin-level access...")

if not MAGPIE_TEST_ADMIN_USERNAME or not MAGPIE_TEST_ADMIN_PASSWORD:
    raise ValueError("Missing test admin credentials to run tests.")

def login_user(username, password):
    _data = {"user_name": username, "password": password}
    _path = MAGPIE_URL + "/signin"
    _resp = requests.post(_path, json=_data, verify=VERIFY_SSL, headers=HEADERS)
    if _resp.status_code != 200:
        raise ValueError("Cannot login with provided test admin credentials.")
    return _resp.cookies

ADMIN_COOKIES = login_user(MAGPIE_TEST_ADMIN_USERNAME, MAGPIE_TEST_ADMIN_PASSWORD)

# test admin-level access with route that is admin only
path = MAGPIE_URL + "/users"
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    raise ValueError("Provided user credentials doesn't have administrative permissions.")

path = "{}/groups/{}".format(MAGPIE_URL, MAGPIE_ANONYMOUS_GROUP)
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    raise ValueError("Could not validate Magpie anonymous group using name: [{}]".format(MAGPIE_ANONYMOUS_GROUP))

## Create a test group and test user that will not have admin access

In [ ]:
print("Create test group and test user...")

TEST_GROUP_NAME = "test-auth-{!s}".format(uuid.uuid4())
data = {"group_name": TEST_GROUP_NAME}
path = MAGPIE_URL + "/groups"
resp = requests.post(path, json=data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    raise ValueError("Could not create test group [{}]".format(data["group_name"]))

TEST_USER_NAME = "test-user-{!s}".format(uuid.uuid4())
data = {"user_name": TEST_USER_NAME, "password": TEST_USER_NAME}
path = MAGPIE_URL + "/groups"
resp = requests.post(path, json=data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    raise ValueError("Could not create test group [{}]".format(data["group_name"]))

## Create specific test resources under THREDDS service

Expected data structure as per:
    [https://github.com/bird-house/birdhouse-deploy/blob/master/birdhouse/scripts/bootstrap-testdata]()

In [ ]:
print("Create test resources under THREDDS service...")

path = MAGPIE_URL + "/services/{}".format(THREDDS_SERVICE)
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    raise ValueError("Could not fetch THREDDS service details [{}].".format(THREDDS_SERVICE))
thredds_res_id = resp.json()["service"]["resource_id"]

def create_or_get_resource(resource_name, resource_type, parent_id):
    _path = MAGPIE_URL + "/resources"
    _data = {"parent_id": parent_id, "resource_type": resource_type, "resource_name": resource_name}
    _resp = requests.post(_path, json=_data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
    _msg = "[name: {}, type: {}, parent: {}].".format(resource_name, resource_type, parent_id)
    if _resp.status_code == 409:
        _path = "{}/{}".format(_path, parent_id)
        _resp = requests.get(_path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
        if _resp.status_code != 200:
            raise ValueError("Could not retrieve resource expected to exist {}.".format(_msg))
        _children = resp.json()["resource"]["children"]
        for _, _child_info in _children.items():
            if _child_info["resource_name"] == resource_name:
                return _child_info
    elif _resp.status_code != 201:
        raise ValueError("Could not create resource {}, unexpected status: [{}]".format(_msg, _resp.status_code))
    else:
        return resp.json()["resource"]
    raise ValueError("Could not create or retrieve resource {}.".format(_msg))

birdhouse_res = create_or_get_resource("birdhouse", "directory", thredds_res_id)
testdata_res = create_or_get_resource("testdata", "directory", birdhouse_res["resource_id"])
secure_res = create_or_get_resource("secure", "directory", testdata_res["resource_id"])
taskmax_name = "tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc"
taskmax_res = create_or_get_resource(taskmax_name, "file", secure_res["resource_id"])

birdhouse_path = birdhouse_res["resource_name"]
birdhouse_catalog = "catalog/{}/catalog.html".format(birdhouse_path)
testdata_path = "{}/{}".format(birdhouse_path, testdata_res["resource_name"])
testdata_catalog = "catalog/{}/catalog.html".format(testdata_path)
secure_path = "{}/{}".format(testdata_path, secure_res["resource_name"])
secure_catalog = "catalog/{}/catalog.html".format(secure_path)
taskmax_path = "{}/{}".format(secure_path, taskmax_res["resource_name"])
taskmax_file = "fileServer/{}".format(taskmax_path)
taskmax_dodsC = "dodsC/{}.html".format(testdata_path)

## Create specific permissions for THREDDS resources to block public access to 'secure' directory

In [ ]:
print("Create permissions on test resources...")

def set_permission(permission, resource, target_type, target_name):
    _res_id = resource["resource_id"]
    _path = "{}/{}/{}/resources/{}/permissions".format(MAGPIE_URL, target_type, target_name, _res_id)
    _data = {"permission": permission}
    _resp = requests.put(_path, json=_data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
    if _resp.status_code not in [200, 201]:
        raise ValueError("Could not set permission [{}] for [{}, {}] over resource [{}, {}]"
                         .format(permission, target_type, target_name, resource["resource_name"], _res_id))


set_permission("browse-allow-recursive", birdhouse_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("read-allow-recursive", birdhouse_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("browse-deny-recursive", secure_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("read-deny-recursive", secure_res, "groups", MAGPIE_ANONYMOUS_GROUP)


## Verify allowed/blocked user access to secured resources

In [ ]:
TEST_USER_COOKIES = login_user(TEST_USER_NAME, TEST_USER_NAME)
ANONYMOUS_COOKIES = {}


def has_access(resource_path, user_cookies, user_name):
    _path = "{}/{}".format(THREDDS_URL, resource_path)
    _resp = requests.get(_path, verify=VERIFY_SSL, cookies=user_cookies)
    if _resp.status_code in [200, 401, 403]:
        is_allowed = _resp.status_code == 200
        print("Access to [{}] for user [{}]: {}".format(_path, user_name, is_allowed))
        return is_allowed
    raise ValueError("Unexpected status code [{}] during access to resource [{}]".format(_resp.status_code, _path))


# admin always has full access regardless of permissions
# avoid potential leak of critical admin username here by using a fill-in value
assert has_access(birdhouse_catalog, ADMIN_COOKIES, "<MAGPIE_TEST_ADMIN_USERNAME>")
assert has_access(testdata_catalog, ADMIN_COOKIES, "<MAGPIE_TEST_ADMIN_USERNAME>")
assert has_access(secure_catalog, ADMIN_COOKIES, "<MAGPIE_TEST_ADMIN_USERNAME>")
assert has_access(taskmax_file, ADMIN_COOKIES, "<MAGPIE_TEST_ADMIN_USERNAME>")
assert has_access(taskmax_dodsC, ADMIN_COOKIES, "<MAGPIE_TEST_ADMIN_USERNAME>")

assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)

assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")


## Verify that top directories are accessible by test user and blocked at lower levels

In [ ]:
# TODO:

set_permission("browse-allow-recursive", secure_res, "groups", MAGPIE_ANONYMOUS_GROUP)

